# Задание 1


In [36]:
from scipy.optimize import linprog
import numpy as np
import pulp

In [37]:
cost = np.array([
    [2, 5, 3],
    [7, 7, 6]
])
stock = np.array([180, 220])
demand = np.array([110, 150, 140])
num_warehouse = 2
num_clients = 3

$x_{ij}$ - сколько забирается со i склада клиенту j  
$$f = \sum_{i,j} cost_{ij} * x_{ij}$$

In [38]:
c = cost.flatten()
print(c) 

[2 5 3 7 7 6]


Для каждого склада количество взятых предметов должно быть меньше, чем на складе:

$$\forall i: \sum_j x_{ij} \leq stock_i$$

In [39]:
A = []
b = []
for i in range(0, num_warehouse):
    A.append([0] * (num_clients * i) + [1] * num_clients + [0] * (num_clients * (num_warehouse - i - 1)))
    b.append(stock[i])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[1 1 1 0 0 0]
 [0 0 0 1 1 1]]
[180 220]


Для каждого клиента количество приобретаемых товаров должно быть больше на единицу, чем спрос:

$$\forall j: \sum_i x_{ij} \geq demand_j$$

Который также:

$$\forall j: - \sum_i x_{ij} \leq -demand_j$$

In [40]:
A = A.tolist()
b = b.tolist()
for j in range(0, num_clients):
    A.append(([0] * j + [-1] + [0] * (num_clients - j - 1)) * num_warehouse)
    b.append(-demand[j])
A = np.asarray(A)
b = np.asarray(b)
print(A)
print(b)

[[ 1  1  1  0  0  0]
 [ 0  0  0  1  1  1]
 [-1  0  0 -1  0  0]
 [ 0 -1  0  0 -1  0]
 [ 0  0 -1  0  0 -1]]
[ 180  220 -110 -150 -140]


In [41]:
linprog(c=c, A_ub=A, b_ub=b)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 1900.0
              x: [ 1.100e+02  0.000e+00  7.000e+01  0.000e+00  1.500e+02
                   7.000e+01]
            nit: 5
          lower:  residual: [ 1.100e+02  0.000e+00  7.000e+01  0.000e+00
                              1.500e+02  7.000e+01]
                 marginals: [ 0.000e+00  1.000e+00  0.000e+00  2.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00]
                 marginals: [-3.000e+00 -0.000e+00 -5.00

Ответ: 110 единиц со склада 1 клиенту 1, 70 единиц со склада 1 клиенту 3,
150 наименований со склада 2 клиенту 2, 70 наименований со склада 2 клиенту 3

In [46]:
# Создаем модель
model = pulp.LpProblem("Optimal_transport_problem", pulp.LpMinimize)

# Создаем переменные для перевозок из складов в торговые центры
x = pulp.LpVariable.dicts("transport",
                         ((i, j) for i in ['склад1', 'склад2'] for j in ['ТЦ1', 'ТЦ2', 'ТЦ3']),
                         lowBound=0,
                         cat='Continuous')

# Добавляем целевую функцию - минимизация затрат на перевозку
model += (
    pulp.lpSum([
        x['склад1','ТЦ1'] * 2 + x['склад2','ТЦ1'] * 7,
        x['склад1','ТЦ2'] * 5 + x['склад2','ТЦ2'] * 7 , 
        x['склад1','ТЦ3'] * 3 + x['склад2','ТЦ3'] * 6 # тарифы из условия
     
    ])
)

# Добавляем ограничения по запасам на складах
model += x['склад1','ТЦ1'] + x['склад1','ТЦ2'] + x['склад1','ТЦ3'] <= 180
model += x['склад2','ТЦ1'] + x['склад2','ТЦ2'] + x['склад2','ТЦ3'] <= 220

# Добавляем ограничения по потребностям торговых центров
model += x['склад1','ТЦ1'] + x['склад2','ТЦ1'] == 110
model += x['склад1','ТЦ2'] + x['склад2','ТЦ2'] == 150
model += x['склад1','ТЦ3'] + x['склад2','ТЦ3'] == 140

# Решаем задачу
model.solve()

# Выводим оптимальный план перевозок
for variable in model.variables():
    print(f"{variable.name} = {variable.varValue}")

print(f"Стоимость оптимального плана: {pulp.value(model.objective)}")


transport_('склад1',_'ТЦ1') = 110.0
transport_('склад1',_'ТЦ2') = 0.0
transport_('склад1',_'ТЦ3') = 70.0
transport_('склад2',_'ТЦ1') = 0.0
transport_('склад2',_'ТЦ2') = 150.0
transport_('склад2',_'ТЦ3') = 70.0
Стоимость оптимального плана: 1900.0
